<a href="https://colab.research.google.com/github/Abhinash200301/Continuous-Time-Networks-for-Adaptive-Process-Control/blob/main/NeuralODE_%2B_LTC_fullyConnected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setups

In [ ]:
!pip install tensorflow==2.13.0
!pip install keras==2.13.1

In [ ]:
!pip install ncps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras import layers, models, regularizers
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [ ]:
!pip install tensorflow_addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.0
    Uninstalling typeguard-4.4.0:
      Successfully uninstalled typeguard-4.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
from ncps import wirings
from ncps.tf import LTC, LTCCell
import seaborn as sns
from tensorflow_addons.metrics import RSquare

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import tensorflow_addons as tfa

In [ ]:
df= pd.read_csv("/content/PostDischarge_Data.csv")

In [ ]:
label=df["label"]
ct=[]
for i in np.unique(label):
  count=0
  for j in range(len(df)):
    if df["label"][j]== i:
      count=count+1
  ct.append(count)

In [ ]:
m=np.unique(label)

In [ ]:
ct

[349, 180, 201, 34, 328, 212, 140, 36, 325, 317, 298, 309]

In [ ]:
l=0
for i in range(len(ct)):
  df1= df[l:l+ct[i]]
  with pd.ExcelWriter(m[i]+".xlsx") as writer:
    df1.to_excel(writer)
  l= ct[i]+l
  print(l)

349
529
730
764
1092
1304
1444
1480
1805
2122
2420
2729


In [ ]:
sequence_length = 5  # Length of the input sequence

In [ ]:
# Get the list of Excel files
a = glob.glob("*.xlsx")

# Lists to store inputs and outputs
train_input = []
train_output = []

# Loop through each file
for j in a:
    dfk = pd.read_excel(j)
    dfm = dfk.drop(columns=['label', 'cycle number', 'Unnamed: 0'])  # Drop unnecessary columns

    # Loop to create sequences based on the specified sequence length
    for i in range(len(dfk) - sequence_length):
        ink = []
        # Collecting the previous 'sequence_length' steps
        for n in range(sequence_length):
            ink.append(list(dfm[i + n:i + n + 1].values[0]))
        train_input.append(ink)

        # Predict the next value after the sequence
        train_output.append(list(dfm[i + sequence_length:i + sequence_length + 1].values[0])[0])

In [ ]:
# Convert lists to numpy arrays
train_input = np.array(train_input)
train_output = np.array(train_output)

In [ ]:
# Ensure train_output is reshaped to (samples, 1)
train_output = train_output.reshape(-1, 1)

In [ ]:
units = 32  # Number of units

In [ ]:
# Wiring: Fully connected Liquid Time Constant (LTC) with 1 motor neuron for output
fc_wiring = wirings.FullyConnected(units, 1)  # 1 output neuron

In [ ]:
model = keras.models.Sequential(
    [
        keras.layers.InputLayer(input_shape=(sequence_length, 121)),
        LTC(fc_wiring, return_sequences=False),  # LTC Layer (returning only the last output)
        # keras.layers.Dense(64, activation="relu"),  # Dense layer with 64 units
        # keras.layers.Dense(1, activation="linear")  # Final output layer (predicting capacity)
    ]
)

# Compile the model with R-squared as the key metric
model.compile(optimizer=keras.optimizers.Adam(0.001), loss='mean_squared_error', metrics=[RSquare()])

In [ ]:
# Clearing any previous session
tf.keras.backend.clear_session()

# # Building and training the new model with Liquid Neural Network
# model = get_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ltc (LTC)                   (None, 1)                 19924     
                                                                 
Total params: 19924 (77.83 KB)
Trainable params: 19924 (77.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# # Set a larger figure size
# sns.set_style("white")
# plt.figure(figsize=(50, 50))  # Adjust the figure size

# # Draw the wiring with labels
# legend_handles = fc_wiring.draw_graph(draw_labels=True)

# # Add a legend
# plt.legend(handles=legend_handles, loc="upper right", bbox_to_anchor=(1.3, 1))

# # Despine and adjust layout
# sns.despine(left=True, bottom=True)
# plt.tight_layout()

# # Show the plot
# plt.show()

## Train

In [ ]:
history = model.fit(train_input, train_output, epochs=250, validation_split=0.1, verbose=1)

Epoch 1/250
76/76 [==============================] - 17s 58ms/step - loss: 866.9719 - r_square: -29.2201 - val_loss: 1094.5244 - val_r_square: -162.2358
Epoch 2/250
76/76 [==============================] - 4s 47ms/step - loss: 837.0709 - r_square: -28.1774 - val_loss: 1060.2727 - val_r_square: -157.1276
Epoch 3/250
76/76 [==============================] - 6s 73ms/step - loss: 810.5453 - r_square: -27.2532 - val_loss: 1034.0662 - val_r_square: -153.2191
Epoch 4/250
76/76 [==============================] - 4s 48ms/step - loss: 787.2040 - r_square: -26.4396 - val_loss: 1010.5867 - val_r_square: -149.7175
Epoch 5/250
76/76 [==============================] - 4s 48ms/step - loss: 766.0949 - r_square: -25.7037 - val_loss: 987.0760 - val_r_square: -146.2111
Epoch 6/250
76/76 [==============================] - 5s 68ms/step - loss: 745.2336 - r_square: -24.9767 - val_loss: 962.8878 - val_r_square: -142.6037
Epoch 7/250
76/76 [==============================] - 4s 51ms/step - loss: 723.9526 - r_sq

In [ ]:
# Save the trained model
model.save("/content/content/my_best_model_liquid.hdf5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from ncps.tf import LTC, LTCCell  # Make sure you import both LTC and LTCCell

In [ ]:
# Step 1: Recreate the wiring structure exactly as it was during training
fc_wiring = wirings.FullyConnected(units, 1)  #1 output neuron (same wiring as during training)

# Model definition using only the LTC architecture (no additional dense layers)
model1 = keras.models.Sequential(
    [
        keras.layers.InputLayer(input_shape=(sequence_length, 121)),
        LTC(fc_wiring, return_sequences=False),  # LTC Layer (returning only the last output)
        # keras.layers.Dense(64, activation="relu"),  # Dense layer with 64 units
        # keras.layers.Dense(1, activation="linear")  # Final output layer (predicting capacity)
    ]

)

# Compile the model with R-squared as the key metric
model1.compile(optimizer=keras.optimizers.Adam(0.001), loss='mean_squared_error', metrics=[RSquare()])

# Step 4: Display the model summary to verify the architecture
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ltc (LTC)                   (None, 1)                 19924     
                                                                 
Total params: 19924 (77.83 KB)
Trainable params: 19924 (77.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Step 5: Load the trained weights from your saved model file
model1.load_weights("/content/Neural ODE + LTC .hdf5")

In [ ]:
# List of battery files
a = glob.glob("*.xlsx")

# Loop through each battery file
for m in a:
    # Load the battery data
    df10 = pd.read_excel(m)
    df101 = df10.drop(columns=['label', 'cycle number', 'Unnamed: 0', "Capacity/mA.h"])

    # Initialize with the first 'sequence_length' capacity values for prediction
    true_capacity = list(df10["Capacity/mA.h"])  # True capacity values for R-squared calculation
    predicted_capacity = true_capacity[:sequence_length]  # First 'sequence_length' true values to start the prediction

    # Iterating through the battery data for predictions with adjustable sequence length
    for i in range(len(df101) - (sequence_length - 1)):
        # Create a sequence of 'sequence_length' steps (the model expects input of shape (sequence_length, 121))
        seq = [list(df101.iloc[i + n]) for n in range(sequence_length)]

        # Insert the true capacity values into the sequence at the appropriate places
        for n in range(sequence_length):
            seq[n].insert(0, predicted_capacity[i + n])  # Insert the corresponding capacity value

        # Convert the sequence into a numpy array and reshape it to (1, sequence_length, 121)
        seq = np.array(seq).reshape(1, sequence_length, 121)

        # Make the prediction using the Liquid Neural Network model
        predict = model1(tf.convert_to_tensor(seq, dtype=tf.float32))

        # Append the predicted capacity value
        predicted_capacity.append(predict.numpy()[0][0])

    # Since predicted_capacity has extra initial values, slice it to match true_capacity length
    predicted_capacity = predicted_capacity[:len(true_capacity)]

    # Compute R-squared value for the predictions
    r_squared = r2_score(true_capacity, predicted_capacity)
    print(f"R-squared for {m.split('.')[0]}: {r_squared}")

    # Save the predicted results for this battery
    results = pd.DataFrame({'True Capacity': true_capacity, 'Predicted Capacity': predicted_capacity})
    with pd.ExcelWriter("/content/content/" + m.split('.')[0] + "_results_with_prediction.xlsx") as writer:
        results.to_excel(writer, index=False)

    # Plot the true and predicted capacity values for this battery
    plt.figure()  # Create a new figure for each battery
    plt.plot(true_capacity, label='True Capacity', color='blue')
    plt.plot(predicted_capacity, label='Predicted Capacity', color='red')
    plt.title(f"True vs Predicted Capacity over Cycles for {m.split('.')[0]}")
    plt.xlabel("Cycle Number")
    plt.ylabel("Capacity (mA.h)")
    plt.legend()
    plt.savefig(f"{m.split('.')[0]}_Capacity_True_vs_Predicted.png")  # Save the plot as a PNG file
    plt.close()  # Close the plot to avoid overlap with the next one

R-squared for 25C03: 0.8486119009931611
R-squared for 45C01: 0.9957811638649253
R-squared for 25C01: 0.9937464434814521
R-squared for 25C04: -6.987134043466714
R-squared for 25C07: 0.9640858227352456
R-squared for 35C01: 0.9973647797751999
R-squared for 25C05: 0.997959188275109
R-squared for 25C02: 0.9623289668388555
R-squared for 25C08: 0.5428551024065432
R-squared for 25C06: 0.8847557171488794
R-squared for 35C02: 0.949461517729117
R-squared for 45C02: 0.8846603395558577
